# Table of Contents
* What Are Color Spaces?
* Simple Segmentation Using Color Spaces
  * Color Spaces and Reading Images in OpenCV
  * Visualizing Whale in RGB Color Space
  * Visualizing Whale in HSV Color Space
  * Picking Out a Range

## What Are Color Spaces?

In the most common color space, **RGB** (Red Green Blue), colors are represented in terms of their red, green, and blue components. In more technical terms, **RGB** describes a color as a tuple of three components. Each component can take a value between 0 and 255, where the tuple (0, 0, 0) represents black and (255, 255, 255) represents white.

RGB is considered an “additive” color space, and colors can be imagined as being produced from shining quantities of red, blue, and green light onto a black background.

Here are a few more examples of colors in RGB:
<hr>
    **Color 	RGB value**
    <hr>
    Red 	255, 0, 0
    <hr>
    Orange 	255, 128, 0
    <hr>
    Blue     0,0,255
    <hr>

RGB is one of the five major color space models, each of which has many offshoots. There are so many color spaces because different color spaces are useful for different purposes.

In the printing world, **CMYK** is useful because it describes the color combinations required to produce a color from a white background. While the 0 tuple in RGB is black, in CMYK the 0 tuple is white. Our printers contain ink canisters of cyan, magenta, yellow, and black.

**HSV** and **HSL** are descriptions of hue, saturation, and brightness/luminance, which are particularly useful for identifying contrast in images. These color spaces are frequently used in color selection tools in software and for web design.

In reality, color is a continuous phenomenon, meaning that there are an infinite number of colors. Color spaces, however, represent color through discrete structures (a fixed number of whole number integer values), which is acceptable since the human eye and perception are also limited. Color spaces are fully able to represent all the colors we are able to distinguish between.

## Simple Segmentation Using Color Spaces
### Color Spaces and Reading Images in OpenCV

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

import cv2

import os
print(os.listdir("../input"))


In [ ]:
img_path='../input/train'
whale_image_list=os.listdir(img_path)
whale_image_list[10:21]

Look at all the color space conversions OpenCV provides

In [ ]:
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]

In [ ]:
len(flags)

In [ ]:
flags[30:40]

**Load an Image**

In [ ]:
whale = cv2.imread(img_path+'/'+whale_image_list[30])
plt.imshow(whale)
plt.show()

### Changing Color-space 

 In fact,**OpenCV** by default reads images in **BGR** format. You can use the function `cv2.cvtColor(input_image, flag)` ( where flag determines the type of conversion) to change color-space.

**Convert from BGR to RGB color-space**

In [ ]:
whale_rgb = cv2.cvtColor(whale, cv2.COLOR_BGR2RGB)
plt.imshow(whale_rgb)
plt.show()

**Convert from BGR to GRAY color-space**

In [ ]:
whale_gray = cv2.cvtColor(whale, cv2.COLOR_BGR2GRAY)
plt.imshow(whale_gray)
plt.show()

**Convert from BGR to HSV color-space**

In [ ]:
whale_hsv = cv2.cvtColor(whale, cv2.COLOR_BGR2HSV)
plt.imshow(whale_hsv)
plt.show()

### Separate the channels of an RGB image 

**R channel**

In [ ]:
red = whale_rgb.copy()
# set blue and green channels to 0
red[:, :, 1] = 0
red[:, :, 2] = 0
# RGB - Red
plt.imshow(red)
plt.show()

**G channel**

In [ ]:
green = whale_rgb.copy()
# set blue and red channels to 0
green[:, :, 0] = 0
green[:, :, 2] = 0
# RGB - Green
plt.imshow(green)
plt.show()


**B channel**

In [ ]:
blue = whale_rgb.copy()
# set green and red channels to 0
blue[:, :, 0] = 0
blue[:, :, 1] = 0
# RGB - Blue
plt.imshow(blue)
plt.show()

### Visualizing Whale in RGB Color Space

**HSV** is a good choice of color space for segmenting by color, but to see why, let’s compare the image in both RGB and HSV color spaces by visualizing the color distribution of its pixels. A 3D plot shows this quite nicely, with each axis representing one of the channels in the color space.

In [ ]:
r, g, b = cv2.split(whale_rgb)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")
pixel_colors = whale_rgb.reshape((np.shape(whale_rgb)[0]*np.shape(whale_rgb)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()
axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

### Visualizing Whale in HSV Color Space

HSV stands for Hue, Saturation, and Value (or brightness), and is a cylindrical color space. The colors, or hues, are modeled as an angular dimension rotating around a central, vertical axis, which represents the value channel. Values go from dark (0 at the bottom) to light at the top. The third axis, saturation, defines the shades of hue from least saturated, at the vertical axis, to most saturated furthest away from the center:

In [ ]:
# whale_hsv = cv2.cvtColor(whale_rgb, cv2.COLOR_RGB2HSV)

In [ ]:
h, s, v = cv2.split(whale_hsv)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

In HSV space, blue’s oranges are much more localized and visually separable. The saturation and value of the blue do vary, but they are mostly located within a small range along the hue axis. This is the key point that can be leveraged for segmentation.

### Picking Out a Range

Let’s threshold whale just based on a simple range of blues.

In [ ]:
# define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

A simple way to display the colors in Python is to make small square images of the desired color and plot them in Matplotlib.

In [ ]:
from matplotlib.colors import hsv_to_rgb
lo_square = np.full((10, 10, 3), lower_blue, dtype=np.uint8) / 255.0
do_square = np.full((10, 10, 3), upper_blue, dtype=np.uint8) / 255.0

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(hsv_to_rgb(do_square))
plt.subplot(1, 2, 2)
plt.imshow(hsv_to_rgb(lo_square))
plt.show()

Once you get a decent color range, you can use cv2.inRange() to try to threshold Whale. inRange() takes three parameters: the image, the lower range, and the higher range. It returns a binary mask (an ndarray of 1s and 0s) the size of the image where values of 1 indicate values within the range, and zero values indicate values outside:

In [ ]:
mask = cv2.inRange(whale_hsv, lower_blue, upper_blue)

In [ ]:
result = cv2.bitwise_and(whale_rgb, whale_rgb, mask=mask)

In [ ]:
plt.figure(figsize=(15,20))
plt.subplot(1, 2, 1)
plt.imshow(mask, cmap="gray")
plt.subplot(1, 2, 2)
plt.imshow(result)
plt.show()

In [ ]:
light_white = (0, 0, 200)
dark_white = (145, 60, 255)

In [ ]:
lw_square = np.full((10, 10, 3), light_white, dtype=np.uint8) / 255.0
dw_square = np.full((10, 10, 3), dark_white, dtype=np.uint8) / 255.0

plt.subplot(1, 2, 1)
plt.imshow(hsv_to_rgb(lw_square))
plt.subplot(1, 2, 2)
plt.imshow(hsv_to_rgb(dw_square))
plt.show()

In [ ]:
mask_white = cv2.inRange(whale_hsv, light_white, dark_white)
result_white = cv2.bitwise_and(whale_hsv, whale_hsv, mask=mask_white)
plt.figure(figsize=(15,20))
plt.subplot(1, 2, 1)
plt.imshow(mask_white, cmap="gray")
plt.subplot(1, 2, 2)
plt.imshow(result_white)
plt.show()

Not bad! Now you can combine the masks. Adding the two masks together results in 1 values wherever there is blue or white,

In [ ]:
final_mask = mask + mask_white

final_result = cv2.bitwise_and(whale_rgb, whale_rgb, mask=final_mask)
plt.figure(figsize=(15,20))
plt.subplot(1, 2, 1)
plt.imshow(final_mask, cmap="gray")
plt.subplot(1, 2, 2)
plt.imshow(final_result)
plt.show()

A Gaussian blur is an image filter that uses a kind of function called a Gaussian to transform each pixel in the image. It has the result of smoothing out image noise and reducing detail.

In [ ]:
blur = cv2.GaussianBlur(final_result, (7, 7), 0)
plt.imshow(blur)
plt.show()

### Segmentation Generalize to Whale’s Relatives

In [ ]:
whales_friends = []
for whale in whale_image_list[:10]:
   friend = cv2.cvtColor(cv2.imread(img_path +'/'+ whale), cv2.COLOR_BGR2RGB)
   whales_friends.append(friend)

In [ ]:
def segment_whale(image):
    ''' Attempts to segment the whale out of the provided image '''

    # Convert the image into HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Set the blue range
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Apply the blue mask 
    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

    # Set a white range
    light_white = (0, 0, 200)
    dark_white = (145, 60, 255)

    # Apply the white mask
    mask_white = cv2.inRange(hsv_image, light_white, dark_white)

    # Combine the two masks
    final_mask = mask + mask_white
    result = cv2.bitwise_and(image, image, mask=final_mask)

    # Clean up the segmentation using a blur
    blur = cv2.GaussianBlur(result, (7, 7), 0)
    return blur

In [ ]:
results = [segment_whale(whale) for whale in whales_friends]

In [ ]:

for i in range(10):
    plt.figure(figsize=(15,20))
    plt.subplot(1, 2, 1)
    plt.imshow(whales_friends[i])
    plt.subplot(1, 2, 2)
    plt.imshow(results[i])
    plt.show()